In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 11.9MB/s 
     |████████████████████████████████| 460kB 62.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


## Import

In [2]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import gensim
from gensim.models import KeyedVectors

from konlpy.tag import Okt, Hannanum, Kkma, Komoran

## Hyperparameter

In [3]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# Hyper-parameters
NUM_LAYERS = 2
D_MODEL = 200 ##word embedding dim
NUM_HEADS = 4 ## D_Model % NUM_HEADS == 0이 되야하므로... D_MODEL / NUM_HEADS = d_k : 한 헤드에서 Q,K,V의 shape (D_MODEL, d_k)
UNITS = 512
DROPOUT = 0.1
EPOCHS = 50
# for data pipelining
BATCH_SIZE = 64
BUFFER_SIZE = 1000

VOCAB_SIZE = 0 # 후에 len(words) 로 바뀜.

# 한 문장에서 단어 시퀀스의 최대 개수
MAX_SEQ = 30

# 정규 표현식 필터
RE_FILTER = re.compile("[\"':;~()]")

## Data Load & Preprocessing

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
wear_data = pd.read_csv("/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/csv_file/wear.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.

(15826, 20)
(8381,) (7445,)


In [6]:
# 고객과 점원의 질,답 쌍으로 만들기
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
             customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))
print(store_arr[-1])
print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)

7301
7301
요즘 파스텔 톤이 유행이에요
요즘 유행하는 색깔이 뭐예요?


In [7]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    # tagger = Komoran()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # [\"':;~()] 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        # sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [8]:
# 형태소분석 수행
question = pos_tag(customer_arr)
answer = pos_tag(store_arr)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q :  신발은 여기 있는 게 다예요?
A : 네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?

Q : 230이요
A : 편하게 신을 수 있는 거 찾으세요?

Q : 네 봄이니까 편하게 신을 수 있는 거
A : 이런 건 어떠세요? 이런 거도 신발 무척 편하거든요

Q : 굽 좀 높은 거 없나요?
A : 봄 상품은 아직 어른 제품이 많이 안나왔습니다

Q : 언제 들어와요?
A : 이번주 지나면 들어올 거예요



In [9]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

#제일 앞에 태그 단어 삽입. 기존 단어들은 뒤로 밀림.
words[:0] = [PAD, STA, END, OOV]

In [10]:
VOCAB_SIZE = len(words)
print("손님과 점원의 말에서 사용된 총 단어의 수 :",len(words))

손님과 점원의 말에서 사용된 총 단어의 수 : 12877


In [11]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [12]:
# 문장을 인덱스로 변환(정수인코딩)
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= MAX_SEQ:
                sentence_index = sentence_index[:MAX_SEQ-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > MAX_SEQ:
                sentence_index = sentence_index[:MAX_SEQ]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (MAX_SEQ - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [13]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (신발 은 여기 있는 게 다예 요)
print(question[0])
x_encoder[0]


 신발은 여기 있는 게 다예요?


array([11152,  8525,  4471,  6872,  4070,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

In [15]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (<START> 신발 은 여기 있는 게 다예 요)
print(answer[0])
x_decoder[0]


네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?


array([    1,  2894,  1508,  2106, 10740,  2157,  7124,  9986,  9880,
        1599,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

In [14]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 입력 출력 (신발 은 여기 있는 게 다예 요 <END>)
print(answer[0])
y_decoder[0]


네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?


array([ 2894,  1508,  2106, 10740,  2157,  7124,  9986,  9880,  1599,
           2,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

## Load Pretrained Embedding model
 - from https://github.com/Kyubyong/wordvectors 's korean(w2v)
 - 200 dim
 - this model trained 330M corpus(tagged by Kkma)


In [16]:
emb_model = gensim.models.Word2Vec.load('/content/drive/My Drive/자연어처리,추천/NLPer/chatbot_pra/ko.bin')
emb_model.save("base_word2vec_200d.model") #기본 full 모델 백업, 더 Train할때 #type: gensim.models.word2vec.Word2Vec

word_vectors = emb_model.wv
word_vectors.save("base_word2vec_200d.wordvectors") #기본 KeyedVecotros 모델 백업, 더 train안시킬때, 로드속도 및 메모리 손실 적음
# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("base_word2vec_200d.wordvectors", mmap='r') #type: gensim.models.keyedvectors.Word2VecKeyedVectors

In [17]:
#일단 추가 학습 없이 해보자.
emb_layer = wv.get_keras_embedding(train_embeddings=False)

## Transformer

In [18]:
# decoder inputs use the previous target as input
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': x_encoder,
        'dec_inputs': x_decoder
    },
    {
        'outputs': y_decoder
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [19]:
dataset

<PrefetchDataset shapes: ({inputs: (None, 30), dec_inputs: (None, 30)}, {outputs: (None, 30)}), types: ({inputs: tf.int64, dec_inputs: tf.int64}, {outputs: tf.int64})>

$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

In [20]:
## scaled dot product Attention
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True) # QK^T

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth) #  QK^T / sqrt(d_k)

  if mask is not None:
    logits += (mask * -1e9) # zero padding token softmax 결과가 0이 나오도록
  
  attention_weights = tf.nn.softmax(logits, axis = -1) # softmax(QK^T / sqrt(d_k))

  output = tf.matmul(attention_weights, value) # softmax(QK^T / sqrt(d_k)) * V

  return output

In [21]:
## multi-head attention
## each head need (scaled_dot_product_attention)
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0 # 128,8

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)
  
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(inputs, shape=(batch_size,-1,self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3]) ##????
  
  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    #linear
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    #split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    #scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    #concatenation of heads
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

    #final linear
    outputs = self.dense(concat_attention)

    return outputs

In [22]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [23]:
#print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

In [24]:
# it handle mask future tokens in a sequence used decoder. and mask pad tokens
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [25]:
#print(create_look_ahead_mask(tf.constant([[1, 2, 0, 4, 5]])))

Positional encoding

since we don't use any rnn, cnn, positional encoding give model position information of words in sentence.

positional encoding vector is added to embedding vector

$$\Large{PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$

In [26]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)
  
  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles #pos/10000^(2i/d_model)

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position = tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model = d_model)
    sines = tf.math.sin(angle_rads[:, 0::2])
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)
  
  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

### Encoder Layer
1. Multi-head attention (with padding mask)
2. 2 dense layers followed by dropout

also has residual connection followd by a layer normalization.

In [27]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query':inputs,
          'key':inputs,
          'value':inputs,
          'mask':padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

### Encoder
1. Input Embedding
2. Positional Encoding
3. `num_layers` encoder layers

Embedding + positional encoding : input

going encoder layers.

output going decoder

In [28]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = emb_layer(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)#??왜 vocab_size가 들어가지?

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units = units,
        d_model = d_model,
        num_heads = num_heads,
        dropout = dropout,
        name = "encoder_layer_{}".format(i),
    )([outputs, padding_mask])
  
  return tf.keras.Model(
      inputs = [inputs, padding_mask], outputs = outputs, name=name)

### Decoder Layer
1. Masked multi-head attention (with look ahead mask and padding mask)
2. Multi-head attention (with padding mask). `value` and `key` is from encoder output. `query` is from Multi-head attention layer output
3. 2 dense layers followed by dropout

also has residual connection followd by a layer normalization.

In [29]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1,1,None), name="padding_mask")

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query' : inputs,
          'key' : inputs,
          'value' : inputs,
          'mask' : look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query' : attention1,
          'key' : enc_outputs,
          'value' : enc_outputs,
          'mask' : padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs = [inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs = outputs,
      name = name)

### Decoder
1. output Embedding
2. Positional Encoding
3. `num_layers` decoder layers

Embedding + positional encoding : input (target)

going decoder layers.

output going final linear layer

In [30]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="decoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1,1,None), name='padding_mask')

  embeddings = emb_layer(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units = units,
        d_model = d_model,
        num_heads = num_heads,
        dropout = dropout,
        name = "decoder_layer_{}".format(i),
    )(inputs = [outputs, enc_outputs, look_ahead_mask, padding_mask])
  
  return tf.keras.Model(
      inputs = [inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs = outputs,
      name = name)

### Transformer
1. encoder
2. decoder
3. final linear layer

In [31]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1,1,None),
      name="enc_padding_mask")(inputs)
  
  #mask future tokens for decoder inputs at 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1,None,None),
      name="look_ahead_mask")(dec_inputs)
  
  #mask encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1,1,None),
      name="dec_padding_mask")(inputs)
  
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

## 모델 생성

In [32]:
tf.keras.backend.clear_session()

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [33]:
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 200)    6771224     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

### Loss function
since target sequences are padded, deal this.

In [34]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_SEQ))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)
  
  mask = tf.cast(tf.not_equal(y_true,0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

### Custom learning rate
use Adam optimizer with custom learning rate
$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$

In [35]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

### Compile Model

In [37]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_SEQ))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

## Train

In [38]:
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
115/115 [==============================] - 3s 28ms/step - loss: 1.5582 - accuracy: 0.0168
Epoch 2/50
115/115 [==============================] - 3s 28ms/step - loss: 1.4197 - accuracy: 0.0333
Epoch 3/50
115/115 [==============================] - 3s 28ms/step - loss: 1.2560 - accuracy: 0.0333
Epoch 4/50
115/115 [==============================] - 3s 28ms/step - loss: 1.1150 - accuracy: 0.0370
Epoch 5/50
115/115 [==============================] - 3s 28ms/step - loss: 1.0658 - accuracy: 0.0419
Epoch 6/50
115/115 [==============================] - 3s 28ms/step - loss: 1.0378 - accuracy: 0.0419
Epoch 7/50
115/115 [==============================] - 3s 28ms/step - loss: 1.0094 - accuracy: 0.0425
Epoch 8/50
115/115 [==============================] - 3s 28ms/step - loss: 0.9804 - accuracy: 0.0432
Epoch 9/50
115/115 [==============================] - 3s 28ms/step - loss: 0.9520 - accuracy: 0.0437
Epoch 10/50
115/115 [==============================] - 3s 28ms/step - loss: 0.9228 - accura

## Predict

In [39]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [40]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [41]:
def evaluate(input_seq):

  input_seq = input_seq.squeeze()
  sentence = tf.expand_dims(input_seq, axis=0)
  output = tf.expand_dims([1], 0)

  for i in range(MAX_SEQ):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, 2):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [42]:
convert_index_to_text(evaluate(make_predict_input('와이프 선물로 뭐가 좋을까요?')).numpy()[1:],index_to_word)

'네 '

In [50]:
convert_index_to_text(evaluate(make_predict_input('아뇨 없어요')).numpy()[1:],index_to_word)

'그러면 평소 좋아하시는 색상 있으세요? '

In [44]:
convert_index_to_text(evaluate(make_predict_input('이거 사이즈 큰거 있어요?')).numpy()[1:],index_to_word)

'네 잠시만요 '

In [45]:
convert_index_to_text(evaluate(make_predict_input('이거 사이즈 230 있어요?')).numpy()[1:],index_to_word)

'네 있어요 '

In [46]:
convert_index_to_text(evaluate(make_predict_input('할인 되요?')).numpy()[1:],index_to_word)

'네 '

In [47]:
# for train data predict
for seq_index in range(0,100):

  print("고객 : ",question[seq_index])
  print("정답점원 :",answer[seq_index])
  print("AI점원 :",convert_index_to_text(evaluate(make_predict_input(question[seq_index])).numpy()[1:],index_to_word))
  print("\n")

고객 :   신발은 여기 있는 게 다예요?
정답점원 : 네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?
AI점원 : 네 성인이나 아동 다 있어요 


고객 :  230이요
정답점원 : 편하게 신을 수 있는 거 찾으세요?
AI점원 : 신발 신어 보시겠어요? 


고객 :  네 봄이니까 편하게 신을 수 있는 거
정답점원 : 이런 건 어떠세요? 이런 거도 신발 무척 편하거든요
AI점원 : 이런 건 어떠세요? 이런 거도 신발 무척 편하거든요 


고객 :  굽 좀 높은 거 없나요?
정답점원 : 봄 상품은 아직 어른 제품이 많이 안나왔습니다
AI점원 : 사이즈는 얼마 신으세요? 


고객 :  언제 들어와요?
정답점원 : 이번주 지나면 들어올 거예요
AI점원 : 이번주 지나면 들어올 거예요 


고객 :  이거는 가죽이에요?
정답점원 : 가죽 아니고 쎄무예요
AI점원 : 아니요 여기 가죽은 없어요 


고객 :  가죽은 얼마예요?
정답점원 : 2만 9천 원입니다
AI점원 : 이거는 2만 9천 원입니다 


고객 :  털 달린 거 저거는 사이즈 있어요?
정답점원 : 230이 없어요  이거 한 번 신어보세요
AI점원 : 230이 없어요 이거 한 번 신어보세요 


고객 :  좀 크네 또 안 들어와요?
정답점원 : 네 이건 다 끝났어요
AI점원 : 어떤 스타일을 원하세요? 


고객 :  가방 매는 거 보고 있어요
정답점원 : 여기 있어요
AI점원 : 여기 있어요 


고객 :  가격이 얼마예요?
정답점원 : 이 종류는 2만 원이고 이 종류는 3만 8천 원이에요
AI점원 : 9만 9천 원이에요 


고객 :  가죽으로 된 거는 없어요?
정답점원 : 가죽은 없고 레자만 있어요
AI점원 : 가죽은 없고 레자만 있어요 


고객 :  레자는 얼마예요?
정답점원 : 5만 5천 원요
AI점원 : 5만 5천 원요 


고객 :  이거는 천이죠?
정답점원 : 네 맞아요
AI점원 : 네 맞아요 


고객 :  이건 얼마예요?
정답점원 : 그것도 5만 5천 원요
AI점원 : 50만 원

## Test data prediction
 - 문장중 14900행부터의 문장 100개에 대한 예측.

In [48]:
test_question_arr = []
test_question = ""

for i in range(14900,wear_data.shape[0]):
        test_question = wear_data.iloc[i].SENTENCE
        test_question_arr.append(test_question)

# test_question_arr = pos_tag(test_question_arr)

In [49]:
# for train data predict
for seq_index in range(0,100):

  print("고객 : ",test_question_arr[seq_index])
  print("AI점원 :",convert_index_to_text(evaluate(make_predict_input(test_question_arr[seq_index])).numpy()[1:],index_to_word))
  print("\n")

고객 :  구제는 뭐예요?
AI점원 : 고객님 이 신발은 사이즈 가방들이 잘 나가요 


고객 :  구제 옷도 있네요?
AI점원 : 네 여기 있습니다 


고객 :  구제품은 다 세탁해서 나온 거예요?
AI점원 : 아직 입고는 안됐고 지금 예약받고있습니다 


고객 :  이건 다 세탁이 된 거죠?
AI점원 : 네 


고객 :  구제품은 세탁해서 파시는 건가요?
AI점원 : 이 신발은 만 이천원이에요 


고객 :  구제품은 다 세탁해서 나오는 거겠죠?
AI점원 : 네, 다 100퍼센트 수제화에요 


고객 :  카드로 결제하면 부가세 별도에요?
AI점원 : 네 


고객 :  카드는 부가세 별도로 붙나요?
AI점원 : 현재 80% 할인하고 있어요 


고객 :  카드 결제하면 부가세 추가되나요?
AI점원 : 네 


고객 :  카드로 하면 부가세 붙어요?
AI점원 : 네 


고객 :  포인트 적립되는 거 뭐 있어요?
AI점원 : 캐시미어 원피스예요 찾는 사이즈가 어떻게 돼요? 


고객 :  이거 적립되죠?
AI점원 : 어떤 스타일을 원하세요? 


고객 :  몸에 딱 붙는 옷인가요?
AI점원 : 쫄바지보다도 이런 바지가 더 붙어요 


고객 :  몸에 딱 붙는 티셔츠 있어요?
AI점원 : 네 여기에 있어요 


고객 :  원피스 중에 몸에 딱 붙는 건 없어요?
AI점원 : 쫄바지보다도 더 붙어요 


고객 :  할인 코너는 어디에요?
AI점원 : 그 제품은 할인 안돼요 


고객 :  할인 코너는 어디에 있어요?
AI점원 : 오른쪽에 있습니다 


고객 :  할인 코너는 따로 없어요?
AI점원 : 네 가능합니다. 


고객 :  할인 코너는 어느 쪽에 있나요?
AI점원 : 오른쪽에 있습니다 


고객 :  영업은 매일 하시나요?
AI점원 : 얇은 자켓은 없어요 


고객 :  내일도 영업해요?
AI점원 : 고객님 왼쪽 장에 진열되어있고 가운데 열 제품들이 원 거예요 


고객 :  매일 영업하시는 건가요?
AI점원 : 네 신상품으로 보여드리겠습니다 


